In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, minimize

plt.rcParams['figure.figsize'] = 15, 8

In [ ]:
data = pd.read_csv('out.csv', encoding='ISO-8859-1')

In [ ]:
plt.plot(data["yhat"])

In [ ]:
def inventory(x):
    a = 0.2
    return 250 * np.log(x + 5)
inv_level = np.linspace(0, 100, 100)
plt.plot(inv_level, inventory(inv_level))

In [ ]:
a = min(data["yhat"]) #min forecast
b = max(data["yhat"]) #max forecast

def elasticity_func(x):
    return b * np.exp(-0.26 * x) * 1e11 + a

pricing_level = np.linspace(100, 120, 100)
plt.plot(pricing_level, elasticity_func(pricing_level))

### So our initial prices (I checked for 100, 110, 120, it works very good for all of them)

In [ ]:
x0 = [120. for _ in range(12)]

In [ ]:
production = data["yhat"]

### bounds (from task description)

In [ ]:
bnds = [(100, 120) for _ in range(12)]

In [ ]:
def profit(prod, to_sell):    
    to_store = prod - to_sell
    inventory(to_store)
    if to_store >= 0:
        return - elasticity_func(to_sell) * to_sell + inventory(to_store) * to_store
    else:
        return -1

### objective function (for minimization) and sales function (for comparison of the results)

In [ ]:
def objective(x, prod):
    return -sum(profit(prod[i], x[i]) for i in range(len(x)))

def sales_function(x):
    return sum([elasticity_func(i) for i in x])

In [ ]:
def constraint(x):
    return [sum(production[:i] - elasticity_func(x[:i])) for i in range(12)]

cons = {'type':'ineq', 'fun':constraint}

### the randomly chosen prices

In [ ]:
print('Initial Objective (Revenue): ' + str(-objective(x0, production)))
print('Initial Sales: ' + str(sales_function(x0)))

In [ ]:
minim = minimize(objective, x0, production, method='SLSQP', bounds=bnds, constraints=cons,options={'maxiter': 100, 'ftol': 1e-06, 'iprint': 1, 'disp': True, 'eps': 1.4901161193847656e-08})

In [ ]:
minim

### So we have a **success**!

In [ ]:
print('Final Objective (Revenue): ' + str(-objective(minim.x, production)))
print('Final Sales: ' + str(sales_function(minim.x)))